# Introduction

We've talked a lot about how to _analyze_ data in Pandas once you have it in the form you want. Let's talk about how to take a dataset, which may or may not be in the final form you want, and manipulate it so that it is.

# Pivot Tables

## What are pivot tables?

Pivot tables are tools typically associated with spreadsheets that can automatically sort, count, total or give the average of the data stored one table, displaying the results in a second table showing the summarized data. Pivot tables are also useful for quickly creating unweighted cross tabulations, or "crosstabs". In Excel, you can set up and change the summary's structure by dragging and dropping fields graphically. This "rotation" or pivoting of the summary table gives the concept its name.

## Pivot tables in Pandas

In Pandas, pandas.pivot_table can be used to create spreadsheet-style pivot tables. It takes a number of arguments - we'll come back to what each of these mean, but suffice it to say: these arguments can be combined and re-combined to do a _wide_ variety of analyses on data. For your reference, the arguments are:

- **data**: A DataFrame object
- **values**: "the values that will be in the _cells_ of your final data frame" - a column or a list of columns to aggregate
- **index**: "the values that will be in the _rows_ of your final data frame" -  a column, Grouper, or array which has the same length as data, or list of them. Keys to group by on the pivot table index. If an array is passed, it is being used as the same manner as column values.
- **columns**: "the values that will be in the _columns_ of your final data frame" - a column, Grouper, array which has the same length as data, or list of them. Keys to group by on the pivot table column. If an array is passed, it is being used as the same manner as column values.
- **aggfunc**: function to use for aggregation, defaulting to numpy.mean
- **margins**: boolean, default False, Add row/column margins (subtotals)

# Detailed walkthrough

## Import libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel("./sales-funnel.xlsx")     # read in excel file

### Exercise: Spend 5 minutes getting to know the data

**Instructions**: Spend 5 minutes getting to know this data. For the first half, write whatever functions you would write yourself. Then, spend 2 minutes comparing what you did with what your neighbor did.

In [3]:
df

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won
5,218895,Kulas Inc,Daniel Hilton,Debra Henley,CPU,2,40000,pending
6,218895,Kulas Inc,Daniel Hilton,Debra Henley,Software,1,10000,presented
7,412290,Jerde-Hilpert,John Smith,Debra Henley,Maintenance,2,5000,pending
8,740150,Barton LLC,John Smith,Debra Henley,CPU,1,35000,declined
9,141962,Herman LLC,Cedric Moss,Fred Anderson,CPU,2,65000,won


In [4]:
df.describe()

,Account,Quantity,Price
count,17.000000,17.000000,17.000000
mean,462254.235294,1.764706,30705.882353
std,259093.442862,1.032558,28444.605609
min,141962.000000,1.000000,5000.000000
25%,218895.000000,1.000000,7000.000000
50%,412290.000000,2.000000,30000.000000
75%,714466.000000,2.000000,40000.000000
max,740150.000000,5.000000,100000.000000


In [5]:
df.corr()

,Account,Quantity,Price
Account,1.000000,0.106514,0.042265
Quantity,0.106514,1.000000,0.563537
Price,0.042265,0.563537,1.000000


In [6]:
df['Quantity'].plot(kind='density')

## Getting to know the data

In [3]:
df.head()

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


In [4]:
df.describe()

,Account,Quantity,Price
count,17.000000,17.000000,17.000000
mean,462254.235294,1.764706,30705.882353
std,259093.442862,1.032558,28444.605609
min,141962.000000,1.000000,5000.000000
25%,218895.000000,1.000000,7000.000000
50%,412290.000000,2.000000,30000.000000
75%,714466.000000,2.000000,40000.000000
max,740150.000000,5.000000,100000.000000


In [7]:
df.dtypes        # type of the variables

Account      int64
Name        object
Rep         object
Manager     object
Product     object
Quantity     int64
Price        int64
Status      object
dtype: object

#### Is each Account assigned a unique Name?

In [8]:
df[['Account', 'Name']].drop_duplicates()       # look at two columns and remove repetitions. looks like acc and name
                                                # are associated one to one.

,Account,Name
0,714466,Trantow-Barrows
3,737550,"Fritsch, Russel and Anderson"
4,146832,Kiehn-Spinka
5,218895,Kulas Inc
7,412290,Jerde-Hilpert
8,740150,Barton LLC
9,141962,Herman LLC
10,163416,Purdy-Kunde
11,239344,Stokes LLC
13,307599,"Kassulke, Ondricka and Metz"


## pivot_table on this data

In [7]:
df.pivot_table(index=["Name"])         # index, order the data by name.    Quantity column is average of each Name-person

,Account,Price,Quantity
Name,,,
Barton LLC,740150.0,35000.0,1.000000
"Fritsch, Russel and Anderson",737550.0,35000.0,1.000000
Herman LLC,141962.0,65000.0,2.000000
Jerde-Hilpert,412290.0,5000.0,2.000000
"Kassulke, Ondricka and Metz",307599.0,7000.0,3.000000
Keeling LLC,688981.0,100000.0,5.000000
Kiehn-Spinka,146832.0,65000.0,2.000000
Koepp Ltd,729833.0,35000.0,2.000000
Kulas Inc,218895.0,25000.0,1.500000


**Explanation:** Names in the rows ("index"), values (all the numeric columns) in the columns by default, their "mean" calculated.

#### Turning "account" into a string

In [9]:
df.Account = df.Account.astype('str')  # acc column is a code for an acc, could be a string instead of float
#df['Account'] can be used as df.Account

In [13]:
df.dtypes

Account     object
Name        object
Rep         object
Manager     object
Product     object
Quantity     int64
Price        int64
Status      object
dtype: object

Now "Account" will not show up when I aggregate.

In [10]:
df.pivot_table(index=["Name"])        # now account# is not included in the pivot

,Price,Quantity
Name,,
Barton LLC,35000.0,1.000000
"Fritsch, Russel and Anderson",35000.0,1.000000
Herman LLC,65000.0,2.000000
Jerde-Hilpert,5000.0,2.000000
"Kassulke, Ondricka and Metz",7000.0,3.000000
Keeling LLC,100000.0,5.000000
Kiehn-Spinka,65000.0,2.000000
Koepp Ltd,35000.0,2.000000
Kulas Inc,25000.0,1.500000


**Bonus exercise:** Turn all of the other columns in the dataframe that are properly categorical into categorical variables, using a _for_ loop!

### More simple aggregations

In [10]:
df.pivot_table(index=['Rep', 'Account'])        # multiple indices.  Organizing the data

Price  Quantity
Rep           Account                    
Cedric Moss   141962    65000.0  2.000000
              163416    30000.0  1.000000
              239344     7500.0  1.000000
Craig Booker  714466    15000.0  1.333333
              737550    35000.0  1.000000
Daniel Hilton 146832    65000.0  2.000000
              218895    25000.0  1.500000
John Smith    412290     5000.0  2.000000
              740150    35000.0  1.000000
Wendy Yule    307599     7000.0  3.000000
              688981   100000.0  5.000000
              729833    35000.0  2.000000

**Explanation:** Both "Rep" and "Account" are included as indices. Price and quantity, the two numeric columns here, are included by default as "values".

What if we only care about price?

In [11]:
df.pivot_table(index=['Rep', 'Account'],
              values=['Price'])               # by default the aggfunction is an average

Price
Rep           Account        
Cedric Moss   141962    65000
              163416    30000
              239344     7500
Craig Booker  714466    15000
              737550    35000
Daniel Hilton 146832    65000
              218895    25000
John Smith    412290     5000
              740150    35000
Wendy Yule    307599     7000
              688981   100000
              729833    35000

#### Changing the "aggfunc"

In [14]:
df.pivot_table(index=['Rep', 'Account'],
              values=['Price'],
              aggfunc=np.sum)              # agg func. sum now

Price
Rep           Account        
Cedric Moss   141962    65000
              163416    30000
              239344    15000
Craig Booker  714466    45000
              737550    35000
Daniel Hilton 146832    65000
              218895    50000
John Smith    412290     5000
              740150    35000
Wendy Yule    307599     7000
              688981   100000
              729833    70000

Comparing the two aggregation functions:

In [13]:
df.pivot_table(index=['Rep', 'Account'],
              values=['Price'],
              aggfunc=[np.sum, np.mean])

sum    mean
                        Price   Price
Rep           Account                
Cedric Moss   141962    65000   65000
              163416    30000   30000
              239344    15000    7500
Craig Booker  714466    45000   15000
              737550    35000   35000
Daniel Hilton 146832    65000   65000
              218895    50000   25000
John Smith    412290     5000    5000
              740150    35000   35000
Wendy Yule    307599     7000    7000
              688981   100000  100000
              729833    70000   35000

**Explanation:** The aggregation functions aggregate for each combination of "Rep" and "Account".

## Exercise (15 minutes)

**Instructions:** Answer the following questions:

What is the most lucrative "Product" category - measured by "highest dollars 'won'"?
Which Rep and Manager have "won" the most in that category (two separate questions)?

In [25]:
df.pivot_table(index=['Product', 'Status'], values=['Price'], aggfunc=np.sum)     # look at price, index product and status.  Average

Price
Product     Status           
CPU         declined   135000
            pending     40000
            presented   60000
            won        230000
Maintenance pending     15000
            won          7000
Monitor     presented    5000
Software    presented   30000

In [35]:
df.pivot_table(index=['Rep', 'Manager', 'Product', 'Status'], values=['Price'])   # index rep, manager, product, status
                                                                  #look at aggfunc=np.sum
# can select a subset of the Dataframe (only status = won) and use a pivot table afterwards

#dfWon = df[df.Status=='won']
#dfWon

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won
9,141962,Herman LLC,Cedric Moss,Fred Anderson,CPU,2,65000,won
13,307599,"Kassulke, Ondricka and Metz",Wendy Yule,Fred Anderson,Maintenance,3,7000,won
14,688981,Keeling LLC,Wendy Yule,Fred Anderson,CPU,5,100000,won


### Answers
Cpu is the most lucrative (highest value of price for won).

Wendy Yule and Fred Anderson won the most in that category (Cpu).

## The "query" function

The query function is an easy, elegant way to select certain values from within a data frame. It works on any data frame, but is especially handy for dataframes with "MultiIndex"es such as those that often arise from pivot tables.

### Complicated (but not "advanced"!) pivot_table

In [30]:
table = df.pivot_table(index=["Manager","Status"],columns=["Product"],values=["Quantity","Price"],
               aggfunc={"Quantity":'count',"Price":[np.sum,np.mean]})           # map quantity to count (# of rows)
table

Price                                         \
                            mean                                    sum   
Product                      CPU Maintenance Monitor Software       CPU   
Manager       Status                                                      
Debra Henley  declined   35000.0         NaN     NaN      NaN   70000.0   
              pending    40000.0      5000.0     NaN      NaN   40000.0   
              presented  30000.0         NaN     NaN  10000.0   30000.0   
              won        65000.0         NaN     NaN      NaN   65000.0   
Fred Anderson declined   65000.0         NaN     NaN      NaN   65000.0   
              pending        NaN      5000.0     NaN      NaN       NaN   
              presented  30000.0         NaN  5000.0  10000.0   30000.0   
              won        82500.0      7000.0     NaN      NaN  165000.0   

                                                     Quantity              \
                                                        count               
Product                 Maintenance Monitor Software      CPU Maintenance   
Manager       Status                                                        
Debra Henley  declined          NaN     NaN      NaN      2.0         NaN   
              pending       10000.0     NaN      NaN      1.0         2.0   
              presented         NaN     NaN  20000.0      1.0         NaN   
              won               NaN     NaN      NaN      1.0         NaN   
Fred Anderson declined          NaN     NaN      NaN      1.0         NaN   
              pending        5000.0     NaN      NaN      NaN         1.0   
              presented         NaN  5000.0  10000.0      1.0         NaN   
              won            7000.0     NaN      NaN      2.0         1.0   

                                          
                                          
Product                 Monitor Software  
Manager       Status                      
Debra Henley  declined      NaN      NaN  
              pending       NaN      NaN  
              presented     NaN      2.0  
              won           NaN      NaN  
Fred Anderson declined      NaN      NaN  
              pending       NaN      NaN  
              presented     1.0      1.0  
              won           NaN      NaN

**Explanation:** Walk through the syntax above and see how the pivot_table function transforms the original table into the one above.

We'd like to subset this, looking only at certain values of the "Manager" and "Product" columns. Here's how we can do that:

In [31]:
table.query('Manager == ["Debra Henley"]')     # SQL kind of syntax.  look at where manager is Debra Henley. single
                                             # quotes outside of the entire query

Price                                        \
                           mean                                   sum   
Product                     CPU Maintenance Monitor Software      CPU   
Manager      Status                                                     
Debra Henley declined   35000.0         NaN     NaN      NaN  70000.0   
             pending    40000.0      5000.0     NaN      NaN  40000.0   
             presented  30000.0         NaN     NaN  10000.0  30000.0   
             won        65000.0         NaN     NaN      NaN  65000.0   

                                                    Quantity              \
                                                       count               
Product                Maintenance Monitor Software      CPU Maintenance   
Manager      Status                                                        
Debra Henley declined          NaN     NaN      NaN      2.0         NaN   
             pending       10000.0     NaN      NaN      1.0         2.0   
             presented         NaN     NaN  20000.0      1.0         NaN   
             won               NaN     NaN      NaN      1.0         NaN   

                                         
                                         
Product                Monitor Software  
Manager      Status                      
Debra Henley declined      NaN      NaN  
             pending       NaN      NaN  
             presented     NaN      2.0  
             won           NaN      NaN

**Explanation:** We typed the value of the column we wanted to use for subsetting, _without quotes_, and then we typed the value to subset on within quotes (a different kind of quotes than the outer quotes, single vs. double). 

We can include multiple values within the brackets as well:

In [32]:
table.query('Status == ["pending","won"] & Manager == ["Debra Henley"]')        # pandas syntax for &, in python 'and' is used

Price                                        \
                         mean                                   sum   
Product                   CPU Maintenance Monitor Software      CPU   
Manager      Status                                                   
Debra Henley pending  40000.0      5000.0     NaN      NaN  40000.0   
             won      65000.0         NaN     NaN      NaN  65000.0   

                                                  Quantity              \
                                                     count               
Product              Maintenance Monitor Software      CPU Maintenance   
Manager      Status                                                      
Debra Henley pending     10000.0     NaN      NaN      1.0         2.0   
             won             NaN     NaN      NaN      1.0         NaN   

                                       
                                       
Product              Monitor Software  
Manager      Status                    
Debra Henley pending     NaN      NaN  
             won         NaN      NaN

## Exercise (10 minutes)

**Note:** You can use "&" to query for multiple conditions.

Using this fact, answer the same question as before returning a table with _only_ the information you need:

Which Rep has "won" the most in the "CPU" category?

Take 5 minutes to work on this on your own, and then 5 minutes with a neighbor.

In [43]:
pivoted_df=df.pivot_table(index=['Rep', 'Product', 'Status'], values = ['Price'], aggfunc=[np.sum])

pivoted_df.query('Status == ["won"] & Product == ["CPU"]')

,,,sum
,,,Price
Rep,Product,Status,
Cedric Moss,CPU,won,65000
Daniel Hilton,CPU,won,65000
Wendy Yule,CPU,won,100000


## Reset_index() and to_csv()

We can use the "reset_index" function to bring these (often annoying) indices in as columns:

In [44]:
df.pivot_table(index=['Product', 'Status'],
              values=['Price'],
              aggfunc=[np.sum])

sum
                        Price
Product     Status           
CPU         declined   135000
            pending     40000
            presented   60000
            won        230000
Maintenance pending     15000
            won          7000
Monitor     presented    5000
Software    presented   30000

In [48]:
df.pivot_table(index=['Product', 'Status'],
              values=['Price'],
              aggfunc=[np.sum]).reset_index()   # creates indices for each line. keeping the columns as before

,Product,Status,sum
,,,Price
0,CPU,declined,135000
1,CPU,pending,40000
2,CPU,presented,60000
3,CPU,won,230000
4,Maintenance,pending,15000
5,Maintenance,won,7000
6,Monitor,presented,5000
7,Software,presented,30000


## to_csv()

to_csv() writes a data frame to a csv file, so that, for example, you can read it in using Excel!

In [46]:
df.pivot_table(index=['Product', 'Status'],
              values=['Price'],
              aggfunc=[np.sum]).reset_index().to_csv('example.csv')   # saves to ipython notebook folder
                                                # using "dot" notation to string commands.

**Explanation:** The string argument inside the parentheses specifies the filename you want to write to.

In [47]:
df.pivot_table(index=['Product', 'Status'],
              values=['Price'],
              aggfunc=[np.sum]).reset_index().to_csv('example.csv', index=False, header=False)      # remove blank spacing

**Explanation:** The header, and index arguments inside the parentheses tell the function not to write the indices and headers to the csv - this gives you a nice, clean dataset.

## "Cheat sheet"

<img src="http://pbpython.com/images/pivot-table-datasheet.png">